In [1]:
import warnings

warnings.filterwarnings("ignore")

In [2]:
import setup
setup.init_django()
from market import services as market_services
from market import tasks as market_tasks
from market.models import Company
import json
from decouple import config

In [3]:
ticker = "AAPL"
name = "Apple"
company, _ = Company.objects.get_or_create(name=name, ticker=ticker)
company.id

2

In [ ]:
market_tasks.sync_historical_stock_data(
    years_ago=1, 
    company_ids=[company.id], 
    use_celery=False, 
    verbose=True
)

In [4]:
import pandas as pd
import setup
from market.models import Company
from market import services as market_services

ticker = "AAPL"
days = 720 

queryset = market_services.get_5min_stock_quotes_queryset(ticker, days=days)

df = pd.DataFrame.from_records(list(queryset.values()))
df['time'] = pd.to_datetime(df['time'])
df.set_index('time', inplace=True)

df = df.rename(columns={
    'open_price': 'Open',
    'high_price': 'High',
    'low_price': 'Low',
    'close_price': 'Close',
    'volume': 'Volume'
})

df[['Open', 'High', 'Low', 'Close']] = df[['Open', 'High', 'Low', 'Close']].astype(float)

df.index = df.index.tz_localize(None)  

df = df[['Open', 'High', 'Low', 'Close', 'Volume']]

print("Cleaned 5-Minute Data Summary:")
print(f"Columns: {df.columns.tolist()}")
print(f"Date Range: {df.index.min()} to {df.index.max()}")
print(f"Total Data Points: {len(df)}")
print(f"\nSample Data:\n{df.head(3)}")

Cleaned 5-Minute Data Summary:
Columns: ['Open', 'High', 'Low', 'Close', 'Volume']
Date Range: 2023-03-02 09:00:00 to 2025-01-25 00:55:00
Total Data Points: 87706

Sample Data:
                       Open    High     Low   Close  Volume
time                                                       
2023-03-02 09:00:00  144.96  144.96  144.23  144.35   11810
2023-03-02 09:05:00  144.34  144.40  144.28  144.40    6443
2023-03-02 09:10:00  144.41  144.41  144.34  144.34    2776
